In [0]:
resources:
  jobs:
    Medisure_Fraud_Monitoring:
      name: Medisure_Fraud_Monitoring
      email_notifications:
        on_start:
          - icon.montalbar@gmail.com
        on_success:
          - icon.montalbar@gmail.com
        on_failure:
          - icon.montalbar@gmail.com
      trigger:
        pause_status: PAUSED
        periodic:
          interval: 1
          unit: HOURS
      tasks:
        - task_key: 01_fraud_detection
          notebook_task:
            notebook_path: /Repos/icon.montalbar@gmail.com/medisure_jen/notebook/01_fraud_detection
            source: WORKSPACE
        - task_key: 02_ml_anomaly_detection
          depends_on:
            - task_key: 01_fraud_detection
          notebook_task:
            notebook_path: /Repos/icon.montalbar@gmail.com/medisure_jen/notebook/02_fraud_ML_anomaly_detection
            source: WORKSPACE
        - task_key: 03_fraud_conditional_routing
          depends_on:
            - task_key: 02_ml_anomaly_detection
          notebook_task:
            notebook_path: /Repos/icon.montalbar@gmail.com/medisure_jen/notebook/03_fraud_conditional_routing
            source: WORKSPACE
        - task_key: alert_generation
          depends_on:
            - task_key: 03_fraud_conditional_routing
          run_if: AT_LEAST_ONE_SUCCESS
          condition_task:
            op: EQUAL_TO
            left: "{{tasks.03_fraud_conditional_routing.values.has_fraud}}"
            right: "true"
        - task_key: 04_alert_generation
          depends_on:
            - task_key: alert_generation
              outcome: "true"
          notebook_task:
            notebook_path: /Repos/icon.montalbar@gmail.com/medisure_jen/notebook/04_fraud_alert_generation
            source: WORKSPACE
        - task_key: alert_false_no_fraud
          depends_on:
            - task_key: alert_generation
              outcome: "false"
          notebook_task:
            notebook_path: /Repos/icon.montalbar@gmail.com/medisure_jen/notebook/04_fraud_alert_generation
            source: WORKSPACE
        - task_key: 05_compliance_reporting
          depends_on:
            - task_key: 04_alert_generation
            - task_key: alert_false_no_fraud
          run_if: AT_LEAST_ONE_SUCCESS
          notebook_task:
            notebook_path: /Repos/icon.montalbar@gmail.com/medisure_jen/notebook/05_fraud_compliance_reporting
            source: WORKSPACE
        - task_key: 06_audit_logging
          depends_on:
            - task_key: 05_compliance_reporting
          notebook_task:
            notebook_path: /Repos/icon.montalbar@gmail.com/medisure_jen/notebook/06_fraud_audit_logging
            source: WORKSPACE
      queue:
        enabled: true
      performance_target: STANDARD
